# 第18章: データベース連携（PDO）

## 学習目標
- PDOの基本概念を理解する
- データベース接続の方法をマスターする
- クエリの実行方法を理解する
- プリペアドステートメントを安全に使用できる
- トランザクション処理を実装できる

---
## 1. PDOとは？

PDO（PHP Data Objects）は、**PHPでデータベースにアクセスするための統一インターフェース**です。

### 🔍 PDOの特徴

- **データベース抽象化**: MySQL, PostgreSQL, SQLiteなど複数のデータベースに対応
- **プレースホルダー**: SQLインジェクションを防ぐための安全なクエリ実行
- **エラーモード**: 例外によるエラーハンドリングが可能
- **OOPインターフェース**: オブジェクト指向で使える

### PDO vs MySQL拡張

 | 特徴 | PDO | MySQL拡張 |
|------|-----|------------|
  | 複数DB対応 | ✅ | ❌（MySQLのみ） |
  | プリペアドステートメント | ✅ | ✅ |
  | エラーハンドリング | ✅（例外） | ❌（警告） |
  | オブジェクト指向 | ✅ | ✅ |
| 非推奨 | ✅（現代では推奨） | ❌（非推奨） |

In [ ]:
<language>php</language><?php

// PDOの基本的な使用例
echo "=== PDOの基本 ===\n";

// データベース接続
try {
    // SQLiteで接続（インストール不要）
    $dsn = 'sqlite::memory:'; // インメモリデータベース
    $username = '';
    $password = '';
    
    $pdo = new PDO($dsn, $username, $password);
    echo "データベースに接続成功\n";
    
    // エラーモードを例外に設定
    $pdo->setAttribute(PDO::ATTR_ERRMODE, PDO::ERRMODE_EXCEPTION);
    
} catch (PDOException $e) {
    echo "接続失敗: {$e->getMessage()}\n";
    exit;
}

// テーブルの作成
try {
    $sql = "CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        age INTEGER,
        created_at DATETIME DEFAULT CURRENT_TIMESTAMP
    )";
    
    $pdo->exec($sql);
    echo "テーブル作成成功\n";
    
} catch (PDOException $e) {
    echo "テーブル作成エラー: {$e->getMessage()}\n";
}

// データの挿入（exec）
try {
    $sql = "INSERT INTO users (name, email, age) VALUES 
        ('山田太郎', 'yamada@example.com', 25),
        ('佐藤花子', 'sato@example.com', 30),
        ('田中次郎', 'tanaka@example.com', 35)
    ";
    
    $result = $pdo->exec($sql);
    echo "{$result}件のデータを挿入\n";
    
} catch (PDOException $e) {
    echo "データ挿入エラー: {$e->getMessage()}\n";
}

// データの取得
try {
    $sql = "SELECT * FROM users";
    $stmt = $pdo->query($sql);
    
    echo "\n=== ユーザーリスト ===\n";
    while ($row = $stmt->fetch(PDO::FETCH_ASSOC)) {
        echo "ID: {$row['id']}, 名前: {$row['name']}, メール: {$row['email']}, 年齢: {$row['age']}\n";
    }
    
} catch (PDOException $e) {
    echo "データ取得エラー: {$e->getMessage()}\n";
}

?>

---
## 2. 接続とエラーハンドリング

データベース接続とエラー処理の実装方法です。

### 接続パラメータの種類

```
// SQLite
sqlite:/path/to/database.db  // ファイルベース
sqlite::memory:              // インメモリ

 // MySQL
mysql:host=localhost;dbname=test;charset=utf8mb4
mysql:host=127.0.0.1;port=3306;dbname=mydb

 // PostgreSQL
pgsql:host=localhost;dbname=test

 // SQL Server
sqlsrv:Server=localhost;Database=test"

 // Oracle
oci:dbname=//localhost:1521/XE"
```

In [ ]:
<language>php</language><?php

// データベース接続クラス
class Database {
    private $pdo;
    private $dsn;
    private $username;
    private $password;
    
    public function __construct($dsn, $username = '', $password = '') {
        $this->dsn = $dsn;
        $this->username = $username;
        $this->password = $password;
    }
    
    public function connect() {
        try {
            $this->pdo = new PDO(
                $this->dsn,
                $this->username,
                $this->password,
                [
                    PDO::ATTR_ERRMODE => PDO::ERRMODE_EXCEPTION,
                    PDO::ATTR_DEFAULT_FETCH_MODE => PDO::FETCH_ASSOC,
                    PDO::ATTR_EMULATE_PREPARES => false,
                ]
            );
            
            echo "データベース接続成功\n";
            return true;
            
        } catch (PDOException $e) {
            echo "接続エラー: {$e->getMessage()}\n";
            return false;
        }
    }
    
    public function getConnection() {
        if (!$this->pdo) {
            $this->connect();
        }
        return $this->pdo;
    }
    
    public function disconnect() {
        $this->pdo = null;
        echo "データベース接続を切断\n";
    }
}

// 異なるデータベースへの接続テスト
echo "=== データベース接続テスト ===\n";

// SQLite
echo "\n--- SQLite（ファイル） ---\n";
$sqliteDb = new Database('sqlite:./test.db');
if ($sqliteDb->connect()) {
    echo "SQLite接続成功\n";
    $sqliteDb->disconnect();
}

// SQLite（インメモリ）
echo "\n--- SQLite（インメモリ） ---\n";
$memoryDb = new Database('sqlite::memory:');
if ($memoryDb->connect()) {
    echo "インメモリDB接続成功\n";
    $memoryDb->disconnect();
}

// MySQL（シミュレーション）
echo "\n--- MySQL ---\n";
$mysqlDsn = 'mysql:host=localhost;dbname=test;charset=utf8mb4';
$mysqlDb = new Database($mysqlDsn, 'root', 'password');
if ($mysqlDb->connect()) {
    echo "MySQL接続成功\n";
    $mysqlDb->disconnect();
}

// PostgreSQL（シミュレーション）
echo "\n--- PostgreSQL ---\n";
$pgsqlDsn = 'pgsql:host=localhost;dbname=test';
$pgsqlDb = new Database($pgsqlDsn, 'postgres', 'password');
if ($pgsqlDb->connect()) {
    echo "PostgreSQL接続成功\n";
    $pgsqlDb->disconnect();
}

?>

---
## 3. クエリの実行方法

PDOでは、クエリの実行方法によって使い分けます。

### クエリ実行の種類

```
// exec() - 結果を返さないクエリ（INSERT, UPDATE, DELETE, CREATE等）
$pdo->exec('INSERT INTO users (name) VALUES ("山田")');

// query() - 結果セットを返すクエリ（SELECT）
$stmt = $pdo->query('SELECT * FROM users');
$rows = $stmt->fetchAll();

// prepare() + execute() - パラメータ付きクエリ（推奨）
$stmt = $pdo->prepare('SELECT * FROM users WHERE id = ?');
$stmt->execute([$id]);
$row = $stmt->fetch();
```

In [ ]:
<language>php</language><?php

// データベース接続
$pdo = new PDO('sqlite::memory:', '', '', [
    PDO::ATTR_ERRMODE => PDO::ERRMODE_EXCEPTION,
    PDO::ATTR_DEFAULT_FETCH_MODE => PDO::FETCH_ASSOC
]);

// テーブル作成
$pdo->exec("CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price REAL NOT NULL,
    stock INTEGER DEFAULT 0,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
)");

// クエリ実行方法テスト
echo "=== クエリ実行方法テスト ===\n";

// 1. exec() の使用例
echo "\n--- exec() ---\n";
$result = $pdo->exec("INSERT INTO products (name, price, stock) VALUES 
    ('PHP入門書', 2980, 100),
    ('MySQL徹底攻略', 3980, 50),
    ('Webデザイン入門', 2500, 80)
");
echo "{$result}件を挿入\n";

// 2. query() の使用例
echo "\n--- query() ---\n";
$stmt = $pdo->query('SELECT * FROM products');
echo "全製品リスト:\n";
while ($row = $stmt->fetch()) {
    echo "{$row['name']}: {$row['price']}円 ({$row['stock']}個)\n";
}

// 3. prepare() + execute() の使用例
echo "\n--- prepare() + execute() ---\n";

// 名前付きプレースホルダー
$stmt = $pdo->prepare('SELECT * FROM products WHERE price > :min_price');
$stmt->execute([':min_price' => 3000]);
echo "3000円以上の製品:\n";
while ($row = $stmt->fetch()) {
    echo "{$row['name']}: {$row['price']}円\n";
}

// 位置プレースホルダー
$stmt = $pdo->prepare('SELECT * FROM products WHERE stock > ?');
$stmt->execute([75]);
echo "在庫75個以上の製品:\n";
while ($row = $stmt->fetch()) {
    echo "{$row['name']}: {$row['stock']}個\n";
}

// プレースホルダーを使った更新
echo "\n--- 更新処理 ---\n";
$stmt = $pdo->prepare('UPDATE products SET stock = stock - 1 WHERE id = ?');
$stmt->execute([1]);
echo "製品1の在庫を減らしました\n";

// 結果を確認
$stmt = $pdo->query('SELECT name, stock FROM products WHERE id = 1');
$row = $stmt->fetch();
echo "{$row['name']}の残り在庫: {$row['stock']}個\n";

?>

---
## 4. フェッチモード

PDOでは、データベースから取得したデータの形式を指定できます。

 ### フェッチモードの種類

| モード | 説明 | 例 |
|--------|------|------|
| `PDO::FETCH_ASSOC` | 連想配列で返す | `['id' => 1, 'name' => '山田']` |
| `PDO::FETCH_NUM` | 数値配列で返す | `[0 => 1, 1 => '山田']` |
| `PDO::FETCH_BOTH` | 両方で返す | 連想配列＋数値配列 |
| `PDO::FETCH_OBJ` | オブジェクトで返す | `$row->id, $row->name` |
| `PDO::FETCH_CLASS` | 指定クラスのインスタンス | `new User()` |
| `PDO::FETCH_COLUMN` | 1列だけ取得 | `[1, 2, 3]` |
| `PDO::FETCH_FUNC` | 各行に関数を適用 | カスタム処理 |
| `PDO::FETCH_KEY_PAIR` | キーと値のペア | `['id' => 'name']` |

In [ ]:
<language>php</language><?php

// データベース準備
$pdo = new PDO('sqlite::memory:', '', '', [
    PDO::ATTR_ERRMODE => PDO::ERRMODE_EXCEPTION
]);

$pdo->exec("CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    age INTEGER
)");

$pdo->exec("INSERT INTO users (name, email, age) VALUES 
    ('山田太郎', 'yamada@example.com', 25),
    ('佐藤花子', 'sato@example.com', 30),
    ('田中次郎', 'tanaka@example.com', 35)
)");

echo "=== フェッチモードの比較 ===\n";

// 1. FETCH_ASSOC
echo "\n--- FETCH_ASSOC ---\n";
$stmt = $pdo->query('SELECT * FROM users LIMIT 1');
$row = $stmt->fetch(PDO::FETCH_ASSOC);
print_r($row);

// 2. FETCH_NUM
echo "\n--- FETCH_NUM ---\n";
$stmt = $pdo->query('SELECT * FROM users LIMIT 1');
$row = $stmt->fetch(PDO::FETCH_NUM);
print_r($row);

// 3. FETCH_BOTH
echo "\n--- FETCH_BOTH ---\n";
$stmt = $pdo->query('SELECT * FROM users LIMIT 1');
$row = $stmt->fetch(PDO::FETCH_BOTH);
print_r($row);

// 4. FETCH_OBJ
echo "\n--- FETCH_OBJ ---\n";
$stmt = $pdo->query('SELECT * FROM users LIMIT 1');
$row = $stmt->fetch(PDO::FETCH_OBJ);
echo "名前: {$row->name}, 年齢: {$row->age}\n";
print_r($row);

// 5. FETCH_CLASS
echo "\n--- FETCH_CLASS ---\n";
class User {
    public $id;
    public $name;
    public $email;
    public $age;
    
    public function getIntroduction() {
        return "{$this->name}（{$this->age}歳）";
    }
}

$stmt = $pdo->query('SELECT * FROM users LIMIT 1');
$user = $stmt->fetch(PDO::FETCH_CLASS, 'User');
echo "{$user->getIntroduction()}\n";
print_r($user);

// 6. FETCH_COLUMN
echo "\n--- FETCH_COLUMN ---\n";
$stmt = $pdo->query('SELECT name FROM users');
$names = $stmt->fetchColumn(0); // 0列目
print_r($names);

// 7. FETCH_ALL
echo "\n--- FETCH_ALL (ASSOC) ---\n";
$stmt = $pdo->query('SELECT * FROM users');
$allUsers = $stmt->fetchAll(PDO::FETCH_ASSOC);
print_r($allUsers);

// 8. FETCH_KEY_PAIR
echo "\n--- FETCH_KEY_PAIR ---\n";
$stmt = $pdo->query('SELECT id, name FROM users');
$keyPairs = $stmt->fetch(PDO::FETCH_KEY_PAIR);
print_r($keyPairs);

?>

---
## 5. トランザクション処理

トランザクションは、**複数のデータベース操作をまとめて扱う**ための仕組みです。

### トランザクションの基本

```
 // トランザクション開始
$pdo->beginTransaction();

 // 処理実行
$pdo->exec('UPDATE accounts SET balance = balance - 1000 WHERE id = 1');
$pdo->exec('UPDATE accounts SET balance = balance + 1000 WHERE id = 2');

// コミット（正常終了）
$pdo->commit();

// ロールバック（異常終了）
$pdo->rollBack();
```

In [ ]:
<language>php</language><?php

// 銀行口座のトランザクション例
echo "=== トランザクション処理 ===\n";

// データベース準備
$pdo = new PDO('sqlite::memory:', '', '', [
    PDO::ATTR_ERRMODE => PDO::ERRMODE_EXCEPTION
]);

// テーブル作成
$pdo->exec("CREATE TABLE accounts (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    balance REAL DEFAULT 0
)");

// 初期データ
$pdo->exec("INSERT INTO accounts (name, balance) VALUES 
    ('山田太郎', 10000),
    ('佐藤花子', 5000)
");

// 口座残高を表示
function displayBalances($pdo) {
    echo "\n--- 口座残高 ---\n";
    $stmt = $pdo->query('SELECT name, balance FROM accounts');
    while ($row = $stmt->fetch(PDO::FETCH_ASSOC)) {
        echo "{$row['name']}: {$row['balance']}円\n";
    }
    echo "合計: " . $pdo->query('SELECT SUM(balance) FROM accounts')->fetchColumn() . "円\n";
}

displayBalances($pdo);

// 正常なトランザクション
echo "\n--- 正常なトランザクション（5000円送金）---\n";
try {
    $pdo->beginTransaction();
    
    // 山田太郎から5000円引き出し
    $pdo->exec("UPDATE accounts SET balance = balance - 5000 WHERE name = '山田太郎'");
    echo "山田太郎: 5000円引き出し\n";
    
    // 佐藤花子に5000円入金
    $pdo->exec("UPDATE accounts SET balance = balance + 5000 WHERE name = '佐藤花子'");
    echo "佐藤花子: 5000円入金\n";
    
    $pdo->commit();
    echo "送金完了！\n";
    
} catch (Exception $e) {
    $pdo->rollBack();
    echo "送金失敗: {$e->getMessage()}\n";
}

displayBalances($pdo);

// 異常なトランザクション
echo "\n--- 異常なトランザクション（存在しない口座に送金）---\n";
try {
    $pdo->beginTransaction();
    
    // 山田太郎から3000円引き出し
    $pdo->exec("UPDATE accounts SET balance = balance - 3000 WHERE name = '山田太郎'");
    echo "山田太郎: 3000円引き出し\n";
    
    // 存在しない田中次郎に送金（失敗）
    $pdo->exec("UPDATE accounts SET balance = balance + 3000 WHERE name = '田中次郎'");
    echo "田中次郎: 3000円入金\n";
    
    $pdo->commit();
    echo "送金完了！\n";
    
} catch (Exception $e) {
    $pdo->rollBack();
    echo "送金失敗: {$e->getMessage()}\n";
    echo "ロールバックしました\n";
}

displayBalances($pdo);

// 保存ポイントの使用
echo "\n--- 保存ポイント（Savepoint） ---\n";
try {
    $pdo->beginTransaction();
    
    // 山田太郎から2000円引き出し
    $pdo->exec("UPDATE accounts SET balance = balance - 2000 WHERE name = '山田太郎'");
    echo "山田太郎: 2000円引き出し\n";
    
    // 保存ポイントを作成
    $savepoint = $pdo->exec("SAVEPOINT before_transfer");
    echo "保存ポイント作成\n";
    
    // 佐藤花子に1000円送金
    $pdo->exec("UPDATE accounts SET balance = balance + 1000 WHERE name = '佐藤花子'");
    echo "佐藤花子: 1000円入金\n";
    
    displayBalances($pdo);
    
    // 保存ポイントにロールバック
    $pdo->exec("ROLLBACK TO before_transfer");
    echo "保存ポイントにロールバック\n";
    
    displayBalances($pdo);
    
    $pdo->commit();
    
} catch (Exception $e) {
    $pdo->rollBack();
    echo "エラー: {$e->getMessage()}\n";
}

?>

---
## 6. リポジトリパターン

リポジトリパターンは、**データベースアクセスをカプセル化**する設計パターンです。

 ### リポジトリパターンの利点

```
- データベース処理の集中化
- テスト容易性（モック可能）
- コードの再利用性向上
- データベースの変更への柔軟性
```

In [ ]:
<language>php</language><?php

// リポジトリパターンの実装例
interface UserRepositoryInterface {
    public function findById($id);
    public function findAll();
    public function save(User $user);
    public function delete($id);
}

// エンティティクラス
class User {
    public $id;
    public $name;
    public $email;
    public $age;
    public $createdAt;
    
    public function __construct($name = '', $email = '', $age = 0) {
        $this->name = $name;
        $this->email = $email;
        $this->age = $age;
        $this->createdAt = new DateTime();
    }
}

// Userリポジトリの実装
class UserRepository implements UserRepositoryInterface {
    private $pdo;
    
    public function __construct(PDO $pdo) {
        $this->pdo = $pdo;
    }
    
    public function findById($id) {
        $stmt = $this->pdo->prepare('SELECT * FROM users WHERE id = :id');
        $stmt->execute([':id' => $id]);
        $data = $stmt->fetch(PDO::FETCH_ASSOC);
        
        if ($data) {
            return $this->createUserFromData($data);
        }
        
        return null;
    }
    
    public function findAll() {
        $stmt = $this->pdo->query('SELECT * FROM users ORDER BY created_at DESC');
        $users = [];
        
        while ($data = $stmt->fetch(PDO::FETCH_ASSOC)) {
            $users[] = $this->createUserFromData($data);
        }
        
        return $users;
    }
    
    public function save(User $user) {
        if ($user->id) {
            // 更新
            $stmt = $this->pdo->prepare('UPDATE users SET name = :name, email = :email, age = :age WHERE id = :id');
            $stmt->execute([
                ':id' => $user->id,
                ':name' => $user->name,
                ':email' => $user->email,
                ':age' => $user->age
            ]);
        } else {
            // 新規作成
            $stmt = $this->pdo->prepare('INSERT INTO users (name, email, age, created_at) VALUES (:name, :email, :age, :created_at)');
            $stmt->execute([
                ':name' => $user->name,
                ':email' => $user->email,
                ':age' => $user->age,
                ':created_at' => $user->createdAt->format('Y-m-d H:i:s')
            ]);
            $user->id = $this->pdo->lastInsertId();
        }
        
        return $user;
    }
    
    public function delete($id) {
        $stmt = $this->pdo->prepare('DELETE FROM users WHERE id = :id');
        return $stmt->execute([':id' => $id]);
    }
    
    private function createUserFromData($data) {
        $user = new User($data['name'], $data['email'], $data['age']);
        $user->id = $data['id'];
        $user->createdAt = new DateTime($data['created_at']);
        return $user;
    }
}

// サービスクラス
class UserService {
    private $userRepository;
    
    public function __construct(UserRepositoryInterface $userRepository) {
        $this->userRepository = $userRepository;
    }
    
    public function registerUser($name, $email, $age) {
        // バリデーション
        if (empty($name) || empty($email)) {
            throw new InvalidArgumentException('名前とメールアドレスは必須です');
        }
        
        if (!filter_var($email, FILTER_VALIDATE_EMAIL)) {
            throw new InvalidArgumentException('メールアドレスが無効です');
        }
        
        // ユーザーの作成と保存
        $user = new User($name, $email, $age);
        return $this->userRepository->save($user);
    }
    
    public function getUsersOlderThan($age) {
        $users = $this->userRepository->findAll();
        return array_filter($users, function($user) use ($age) {
            return $user->age > $age;
        });
    }
}

// データベース準備
$pdo = new PDO('sqlite::memory:', '', '', [
    PDO::ATTR_ERRMODE => PDO::ERRMODE_EXCEPTION
]);

$pdo->exec("CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    age INTEGER,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
)");

echo "=== リポジトリパターン ===\n";

// リポジトリとサービスの使用
$userRepository = new UserRepository($pdo);
$userService = new UserService($userRepository);

// ユーザー登録
echo "\n--- ユーザー登録 ---\n";
$user1 = $userService->registerUser('山田太郎', 'yamada@example.com', 25);
echo "登録ユーザー: {$user1->name} (ID: {$user1->id})\n";

$user2 = $userService->registerUser('佐藤花子', 'sato@example.com', 30);
echo "登録ユーザー: {$user2->name} (ID: {$user2->id})\n";

// 全ユーザー取得
echo "\n--- 全ユーザー一覧 ---\n";
$allUsers = $userRepository->findAll();
foreach ($allUsers as $user) {
    echo "{$user->name} ({$user->age}歳) - {$user->email}\n";
}

// 年齢でフィルタリング
echo "\n--- 25歳以上のユーザー ---\n";
$olderUsers = $userService->getUsersOlderThan(25);
foreach ($olderUsers as $user) {
    echo "{$user->name} ({$user->age}歳)\n";
}

// ユーザー情報の更新
echo "\n--- ユーザー情報更新 ---\n";
$user1->age = 26;
$updatedUser = $userRepository->save($user1);
echo "{$updatedUser->name}の年齢を{$updatedUser->age}歳に更新\n";

?>

---
## 📝 実践問題

### 問題1: PDOベースのCRUDシステム

完全なCRUD機能を持つシステムを作成してください。

   ```
要件:
- `Product`エンティティ
- `ProductRepository`（CRUD操作）
- `ProductService`（ビジネスロジック）
- 在庫管理機能
- 在庫不足時の警告
```

### 問題2: 多対多のリレーション

多対多のリレーションを扱うシステムを作成してください。

 ```
要件:
- `User`と`Role`の多対多リレーション
- ユーザーに複数のロールを割り当て可能
- ロールごとの権限管理
- リポジトリパターンを使用
```

### 問題3: キャッシュシステム

データベース結果をキャッシュするシステムを作成してください。

 ```
要件:
- キャッシュ用テーブル
- TTL（Time To Live）機能
- キャッシュヒット/ミットの判別
- 自動キャッシュ更新
```

### 問題4: バッチ処理

大量のデータを処理するバッチシステムを作成してください。

 ```
要件:
- ページネーションを使った大量データ取得
- プログレス表示
- 中断と再開機能
- エラーログ
```

### 問題5: マイグレーションシステム

データベースマイグレーションシステムを作成してください。

 ```
要件:
- マイグレーションファイルの管理
- up/down/rollback機能
- バージョン管理
- トランザクションでのマイグレーション
```

### 問題6: リードレプリカ

マスタースレーブ構成を実装してください。

 ```
要件:
- マスターDB：書き込み用
- スレーブDB：読み込み用
- 自動フェイルオーバー
- 読み書きの自動切り替え
```

---
## まとめ

### ✅ ポイントチェック

- [ ] PDOの基本概念を理解した
- [ ] データベース接続の方法をマスターした
- [ ] クエリの実行方法を理解した
- [ ] プリペアドステートメントを安全に使用できる
- [ ] トランザクション処理を実装できる
- [ ] リポジトリパターンを理解した

### 🚀 次のステップ

これでPHPの基礎から応用まで学習完了です！次は実際のWebアプリケーション開発に挑戦しましょう。

---

**関連リソース**
- [PDO - Manual](https://www.php.net/manual/ja/book.pdo.php)
- [PHP: The Right Way - Database](https://phptherightway.com/#database)
- [Repository Pattern](https://designpatternsphp.readthedocs.io/en/latest/Structural/Repository/README.html)
- [Active Record vs Repository Pattern](https://stackoverflow.com/questions/3060191/repository-pattern-vs-active-record-pattern)